## 多智能体强化学习--相关笔记&梳理

#### 多智能体存在问题：

1. **维度爆炸：** 状态空间变大，联结动作空间维度非常大，计算复杂

2. **目标奖励确定困难：** _多智能体系统中每个智能体的任务可能不同，但是彼此之间又相互耦合影响。奖励设计的优劣直接影响学习到的策略的好坏_

3. **不稳定性：** 在多智能体系统中，多个智能体是同时学习的。当同伴的策略改变时，每个智能体自身的最优策略也可能会变化，这将对算法的**收敛性**带来影响。

4. **探索-利用：** 探索不光要考虑自身对环境的探索，也要对同伴的策略变化进行探索，可能打破同伴策略的平衡状态。每个智能体的探索都可能对同伴智能体的策略产生影响，这将使算法很难稳定，学习速度慢。

##### 🔹维度爆炸
解决方案:
1. 状态空间优化

    对于第一点，状态空间简化一般由**特征工程**来处理，一般方法:
    - PCA(Principal Component Analysis)常用于高维数据的降维
    - LDA(线性判别算法)算法

2. 联合动作空间优化
    - **Mean Field MARL算法:** 主要解决的是集中式多智能体强化学习中，$Q(s,a)$转换为只包含邻居之间相互作用的形式$$Q_{j}(s, a)=\frac{1}{N_{j}} \sum_{k \in N(j)} Q_{j}\left(s, a_{j}, a_{k}\right)$$其中$N(j)$表示智能体j邻居智能体的标签集<br>
    核心:近似保留邻居之间的交互，去掉了非邻居之间的交互
---
##### 🔹目标奖励函数
解决方案:
- **Curiosity Driven 好奇心驱动:** 

    使用内在奖励鼓励agent探索更陌生的状态，平衡探索与利用，本质上是提高了样本的利用效率，主要分为两类:
    - 基于状态计数<br>
    [Count-based exploration with neural density models](https://arxiv.org/pdf/1703.01310.pdf)<br>
    [Exploration: A study of count-based exploration for deep reinforcement learning](https://arxiv.org/abs/1611.04717)<br>
    - 基于状态预测误差的方法<br>
    [Incentivizing exploration in reinforcement learning with deep predictive models](https://arxiv.org/pdf/1507.00814v2)<br>
    [ICM（Curiosity-driven Exploration by Self- supervised Prediction）](https://arxiv.org/pdf/1705.05363.pdf)<br>

- **Hindsight Experience Replay（HER）无奖励样本学习:** 
    
    一般的强化学习方法对于无奖励的样本几乎没有利用，HER的思想就是从无奖励的样本中学习。HER建立在多目标强化学习的基础上，将失败的状态映射为新的目标<br>[HER文章](https://arxiv.org/abs/1707.01495)

- **Priority Experience Replay 采样率挂钩td_error:** 

    PER是DQN中最有效的改进手段之一，通过改变样本采样的概率来提高样本利用率和训练速度。Prioritized DQN中使用与td-error大小成正比的采样概率，在稀疏奖励中还可以对正样本采用更高的采样概率，防止在大量无奖励样本中“淹没”<br>[Prioritized DQN](https://arxiv.org/abs/1511.05952)

---
##### 🔹不稳定性
- **PPO:常用cliping截断**

    常用的方法类似于PPO的cliping截断--即为限制td_error的范围，非常好用，有相关论文 <br>[Deep Reinforcement Learning and the Deadly Triad](https://arxiv.org/abs/1812.02648)

- **DDPG: Soft target update 软更新**

    $$\omega^{-} \leftarrow \tau \omega+(1-\tau) \omega^{-}$$
    软更新让参数的更新不至于发生剧变，从而稳定了训练

- **DDQN: 使用目标网络**

    有优化操作会传播高估误差：DDQN（Double DQN）更加稳定，因为最优化操作会传播高估误差，所以同时训练两个Q network并选择较小的Q值用于计算TD-error，降低高估误差
    
- **Hierarchical Reinforcement Learning 分层强化学习**

    使用多层次的结构来学习不同层次的策略。把分层次解决复杂问题，把复杂劳动分解成多个简单劳动，使得算法更好收敛

----
##### 🔹探索-利用

- **在探索不足的情况下有鼓励探索的好奇心机制**

    [Exploration Strategies in Deep Reinforcement Learning](https://lilianweng.github.io/posts/2020-06-07-exploration-drl/)<br>
    [Curiosity-driven reinforcement learning with homeostatic regulation](https://arxiv.org/abs/1801.07440)

探索策略可以分为两类：有向的和无向的。

有向的探索策略考虑之前的历史，而无向的策略随机探索，不考虑之前的学习。这一节讨论三个部分:<br>
一是无向的探索，二是基于观测学习过程的有向探索，三是其他基于学习模型的有向探索

1. **Undirected Exploration 随机分布探索**

    动作通过随机分布生成，不考虑学习过程。Thrun认为无向的探索不如有向的探索。这里介绍三种无向探索策略：
    - 随机探索（random exploration）
    - 半均匀分布探索（semi-uniform distributed exploration）
        $$P(a)=\left\{\begin{array}{ll}P_{\text {best }}+\frac{1-P_{\text {best }}}{\# \text { of actions }} & , \text { if } a \text { maximizes } f(a) \\ \frac{1-P_{\text {best }}}{\# \text { of actions }} & , \text { otherwise }\end{array}\right.$$
        - 这种策略的基本思想是，给最佳的动作一个较高的概率，而其他动作则均匀地分配剩余的概率
        - 公式中的 $P_{\text{best}}$ 表示最佳动作的概率。当 $P_{\text{best}} = 1$ 时，策略完全利用已知的最佳动作；当 $P_{\text{best}} = 0$ 时，策略进行完全随机的探索。
    - 玻尔兹曼分布探索（Boltzmann distributed exploration） 
        $$P(a)=\frac{e^{f(a) \cdot \theta^{-1}}}{\sum_{a^{\prime} \in \text { Actions }} e^{f\left(a^{\prime}\right) \cdot \theta^{-1}}}$$
        - 这种策略基于动作的效用值（或回报）来分配概率。每个动作的概率与其效用值成指数比例。这里的 $\theta$ 是一个温度参数，用于控制探索和利用之间的权衡。
        - 当 $\theta \rightarrow 0$ 时，策略完全利用最佳动作；当 $\theta \rightarrow \infty$ 时，策略进行完全随机的探索
    随机探索效果不好，姑且不考虑，最终结论为：
    > 半均匀分布探索要比完全随机探索和玻尔兹曼分布探索都更好，半均匀分布探索策略直接为最佳动作分配一个预先定义的概率，而玻尔兹曼分布策略则基于动作的效用值来动态调整概率
    
2. **Directed Exploration 有向探索策略**
    
    在这里介绍四种基本的有向探索策略：
    - Counter-based exploration基于计数的探索
        $$\begin{array}{l}\text { eval }_{c}(a)=\alpha \cdot f(a)+\frac{c(s)}{E[c \mid s, a]} \\ E[c \mid s, a]=\sum P_{s \rightarrow s^{\prime}}(a) \cdot c\left(s^{\prime}\right)\end{array}$$
        动作用利用值和探索项的和来评估，这里的探索项是当前状态的计数和状态的期望计数的商
    - Counter-based exploration with decay基于计数的带衰减的探索
        $$c(s) \leftarrow \lambda \cdot c(s) \quad \forall s$$
        加了一个小于等于1的参数
    - Counter/Error-based exploration
        $$eval _{c}(a)=\alpha \cdot f(a)+\frac{c(s)}{E[c \mid s, a]}+\beta \cdot E\left[\Delta \hat{V}_{\text {last }} \mid s, a\right]$$
    - Recency-based exploration基于近因的探索
        $$\operatorname{eval}_{r}(a)=\alpha \cdot f(a)+\sqrt{E[\rho \mid s, a]}$$

3. **其他有向探索**
    - Competence Maps：使用一个辅助的数据结构来估计agent相信的程度，相信在哪个区域自己有足够的知识来做出好的决策。这个估计用来探索世界，选择的动作让agent去它认为有最小competence的区域。
    - Interval Estimation：学习的是动作价值以置信概率落在上下界中，但这种方法在实际中是有问题的，因为方法的复杂以及统计假设不能满足
    - Multiple-Hypothesis Heuristics：对Q-learning算法的一个修正，是计算可行的

---

##### 附录:详细讨论四种有向探索策略
1. **Counter-based exploration 基于计数的探索**

    这种策略主要依赖于为每一个状态-动作对 $(s, a)$ 计数。这个计数用于估计我们多少次访问了某一个状态并采取了某个动作。

    $$\text { eval }_{c}(a) = \alpha \cdot f(a) + \frac{c(s)}{E[c \mid s, a]}$$

    这里，$\alpha \cdot f(a)$ 是一个基于动作 $a$ 的价值函数。探索部分由 $\frac{c(s)}{E[c \mid s, a]}$ 表示，其中 $E[c \mid s, a]$ 是在状态 $s$ 并采取动作 $a$ 之后的预期计数。这种方法鼓励代理在未经常访问的状态中采取行动。

2. **Counter-based exploration with decay 基于计数的带衰减的探索**

    这种策略的思路是随着时间的推移逐渐减少状态的计数：

    $$c(s) \leftarrow \lambda \cdot c(s) \quad \forall s$$

    其中，$\lambda$ 是一个衰减因子，它通常小于等于1。这意味着早期的访问将逐渐失去其重要性，使得代理有更大的倾向去重新探索那些它很久没有访问的状态。

3. **Counter/Error-based exploration**

    这种策略不仅考虑访问计数，还考虑价值函数的估计误差。当我们对某个状态-动作对的价值函数的估计越不确定时，我们应该更加倾向于探索这个状态-动作对。

    $$\text{eval}_{c}(a) = \alpha \cdot f(a) + \frac{c(s)}{E[c \mid s, a]} + \beta \cdot E\left[\Delta \hat{V}_{\text {last }} \mid s, a\right]$$

    其中，$E\left[\Delta \hat{V}_{\text {last }} \mid s, a\right]$ 表示在状态 $s$ 采取动作 $a$ 后价值函数的预期变化。

4. **Recency-based exploration 基于近因的探索**

    这种策略的核心思想是优先考虑那些最近没有访问过的状态-动作对。其探索度量基于最后一次访问该状态-动作对的时间。

    $$\text{eval}_{r}(a) = \alpha \cdot f(a) + \sqrt{E[\rho \mid s, a]}$$

    其中，$E[\rho \mid s, a]$ 表示自从上次访问状态 $s$ 并采取动作 $a$ 以来的时间。

---

## 多智能体相关优化算法

1. 实现星际争霸100%胜率<br>
[微信公众号-原文](https://mp.weixin.qq.com/s/C_bNa42FdR5xLRcbSLXSCg)<br>
[知乎-原作者-增加对实验环境描述](https://zhuanlan.zhihu.com/p/660794690)<br>
[HPN实现-星际争霸env](https://github.com/tjuHaoXiaotian/pymarl3)

2. ASN动作语义网络<br>
[ASN:知乎详细解读](https://zhuanlan.zhihu.com/p/199072017)<br>
[ASN:严格公式解读](https://mayi1996.top/2020/08/14/Action-Semantics-Network-Considering-the-Effects-of-Actions-in-Multiagent-Systems/)<br>
[ASN论文原文:Action Semantics Network: Considering the Effects of Actions in Multiagent Systems](http://arxiv.org/abs/1907.11461)<br>

3. HPN超网络<br>
[HPN：利用超网络实现「置换不变性」「置换同变性」](https://zhuanlan.zhihu.com/p/613723150)<br>
[HPN论文原文:Breaking the Curse of Dimensionality in Multiagent State Space: A Unified Agent Permutation Framework](https://arxiv.org/abs/2203.05285)

附录:高价值文章来源<br>
1. 知乎-XuanAxuan 大量总结了深度强化学习的论文解读 
[深度强化学习笔记汇总](https://zhuanlan.zhihu.com/p/546276372)<br>

2. 星际争霸100%胜率--原作者:专攻多智能体强化学习 
[多智能体相关研究](https://www.zhihu.com/people/tju-hao-xiao-tian/answers)<br>

3. 网易伏羲实验室--多智能体相关前沿进展
[专注于大模型&多智能体](https://www.zhihu.com/people/wang-xiao-wei-64-66)<br>


### ASN动作语义网络

此前的多智能体强化学习算法都没有明确地考虑智能体之间的 _动作语义_，即 **智能体选择不同的动作会对其他智能体有不同的影响** 。在本文中，明确提出了一种新的网络结构，称为 **动作语义网络(Action Semantics Network)**

#### 背景
每个agent的动作集可以自然地分为两种类型:
- $A_in$一种包含直接影响环境信息和智能体自己私有属性的动作
- $A_out$另一种包含直接影响其他智能体的动作

将不同的动作可能对其他智能体产生不同影响的属性称为 **动作语义(Action Semantics)**，相当于将状态分开处理，动作value更依赖于observation的 $i \rightarrow j$，则其他均为噪声

以前的工作通常使用所有可用的信息来估计所有动作的价值，这有可能很低效，于是ASN不是将一个智能体的observation输入到一个网络中，而让observation由几个子模块组成，这些子模块根据动作的语义将智能体observation的不同部分作为输入，从而获得更准确的估计

---

#### 结构
![](https://s1.ax1x.com/2020/08/14/dCTiFg.png)

主要改进:一是把动作（action）划分开，二是把观测量（observation）划分开

1. 动作划分:<br>对于这两种动作，用两套不同网络去评估，可以求Q也可以求分布
    - $A_{in}^i$只影响环境和自己的动作，比如移动
    - $A_{out}^i$影响其他智能体的动作，比如协助任务中的交流，竞争任务中的攻击

2. observation划分:<br>
    $$o_{t}^{i}=\left(o_{t}^{i, e n v}, m_{t}^{i}, o_{t}^{i, 1}, \cdots, \quad o_{t}^{i, n}\right)$$
    - $o_{t}^{i, e n v}$有关环境的observation
    - $m_{t}^{i}$自己的固有属性。比如生命值，攻击力……
    - $o_{t}^{i, n}$和其他智能体有关的observation

因此输出也被划分为: 
- **直接影响环境的动作（橘色）**
- **会影响其他智能体的动作（蓝色）**

网络也被分为:<br>
- **与环境交互的部分（橘色的O2A i）**

    橘色网络以全部的observation作为输入，第一层网络O2E输出一个编码$e_t^i$，然后第二层网络E2A输出对动作的评估
- **与其他智能体交互的部分（蓝色的O2A i,j）**

    蓝色网络以observation中和智能体j有关的部分$o_t^{i,j}$作为输入，全部observation的编码$e_t^i$作为补充输入，输出 **「智能体i对智能体j的评估」**

而输出的形式，则根据base算法:
- 如果是value-base的算法，输出的评估是Q值
    $$Q\left(o_{t}^{i}, a_{t}^{i}\right)=f a\left(e_{t}^{i}, a_{t}^{i}\right),Q\left(o_{t}^{i}, a_{t}^{i, j}\right)=\mathcal{M}\left(e_{t}^{i}, e_{t}^{i, j}\right)$$
    其中$\mathcal{M}\left(e_{t}^{i}, e_{t}^{i, j}\right)$表示的是两个embedding特征向量的内积|
- 如果是policy-base的算法，输出对应的概率分布$$
    $$\pi\left(a_{t}^{i} \mid o_{t}^{i}\right)=\frac{\exp \left(f a\left(e_{t}^{i}, a_{t}^{i}\right)\right)}{Z^{\pi_{i}}\left(o_{t}^{i}\right)}, \pi\left(a_{t}^{i, j} \mid o_{t}^{i}\right)=\frac{\exp \left(\mathcal{M}\left(e_{t}^{i}, e_{t}^{i, j}\right)\right)}{Z^{\pi_{i}}\left(o_{t}^{i}\right)}$$

> 其他形象化解释在网页上 [以SMAC中的3m（三枪兵对战任务）来举例](https://zhuanlan.zhihu.com/p/199072017)

---

#### ASN扩展

##### 将ASN算法与其他算法结合
- ASN-PPO：使用ASN替换PPO中的策略网络
- ASN-QMIX：使用ASN替换QMIX中的Q网络

#### ASN-PPO
注意，只展示了ASN的右侧部分（不包括ASN的左侧部分 $O 2 A^{i}$）作为不同的变种
![ASN-PPO](https://pic3.zhimg.com/80/v2-0fa11e523ce1ab5b530a7a878d192e7a_1440w.webp)

$A_{t}$ 是在时间步 $t$ 的优势函数。PPO使用约束和优势估计来重新定义优化问题为：
$$
\max _{\theta^{i}} \mathbb{E}_{t}\left[r_{t}\left(\theta^{i}\right) A_{t}\left(o_{t}^{i}, a_{t}^{i}\right)\right]
$$
其中，$r_{t}\left(\theta^{i}\right)$ 是概率比率 $\frac{\pi_{\theta^{i}}\left(a_{t}^{i} \mid o_{t}^{i}\right)}{\pi_{\theta_{\text{old}}^{i}}\left(a_{t}^{i} \mid o_{t}^{i}\right)}$，$\theta_{\text{old}}^{i}$ 是更新前的策略参数。

然后，在ASN-PPO中，$r_{t}\left(\theta^{i}\right)$ 可以通过替代等式2重写为：
$$
r_{t}\left(\theta^{i}\right) = \left\{\begin{array}{ll}
\text{某种由}\ \mathcal{A}_{\text{in}}^{i}\ \text{决定的形式} \\
\text{某种由}\ \mathcal{A}_{\text{out}}^{i}\ \text{决定的形式} \\
\end{array}\right.
$$
最后，ASN-PPO在每次迭代中按照PPO来最大化目标函数。
而$r_{t}$可以被重写为
$$
r_{t}\left(\theta^{i}\right)=\left\{\begin{array}{ll}\frac{\exp \left(f a\left(\boldsymbol{e}_{t}^{i}, a_{t}^{i} ; \theta^{i}\right)\right)}{\exp \left(f a\left(\boldsymbol{e}_{t}^{i}, a_{t}^{i} ; \theta^{i}\right)\right.} \frac{Z^{\pi_{i}}\left(o_{t}^{i} ; \theta_{\text {old }}^{i}\right)}{\left.Z^{\pi_{i}}\right)} & \text { if } a_{t}^{i} \in \mathcal{A}_{\text {in }}^{i} \\ \frac{\exp \left(\mathcal{M}\left(\boldsymbol{e}_{t}^{i}, \boldsymbol{e}_{t}^{i, j} ; \theta^{i}\right)\right)}{\exp \left(\mathcal{M}\left(\boldsymbol{e}_{t}^{i}, \boldsymbol{e}_{t}^{i, j} ; \theta_{\text {old }}^{i}\right)\right)} \frac{Z^{\pi_{i}}\left(o_{t}^{i} ; \theta_{\text {old }}^{i}\right)}{Z^{\pi_{i}}\left(o_{t}^{i} ; \theta^{i}\right)} & \text { if } a_{t}^{i} \in \mathcal{A}_{\text {out }}^{i}\end{array}\right.
$$
这意味着输出的策略本身就应该是`log_prob`


代码实现

为了简单处理，没有用单独的Embedding层去转为特征向量，还需要修改更新函数以使用新的$r_t$计算方式:
1. **策略网络修改**：
    - 将输出动作分为$A_{in}$和$A_{out}$,可以为每种动作类型设置一个独立的线性层
    - 观测分为关于环境、关于自己和关于其他智能体的部分,可以根据需要对这些部分进行不同的处理

2. **修改更新函数**：
    
    需要计算新的$r_t$。具体计算方式取决于动作是否属于$A_{in}$还是$A_{out}$

3. **观测的区分**：
    
    在ASN中，观测被分为关于环境的、关于自己的和关于其他智能体的。这意味着在策略和价值网络中，我们需要考虑这些不同的观测输入
    
但是现在actor_loss要怎么计算？<br>如果使用PPO截断，将会产生`surr1_in，surr2_in，surr1_out，surr2_out`，不能再像之前那样`actor_loss = torch.mean(-torch.min(surr1, surr2))`了

这里仍然为了简单起见，直接使用平均值，可能之后还要调整参数:
```python
surr1_in = ratio_in * advantage
surr2_in = torch.clamp(ratio_in, 1 - self.eps, 1 + self.eps) * advantage

surr1_out = ratio_out * advantage
surr2_out = torch.clamp(ratio_out, 1 - self.eps, 1 + self.eps) * advantage

actor_loss_in = torch.mean(-torch.min(surr1_in, surr2_in))
actor_loss_out = torch.mean(-torch.min(surr1_out, surr2_out))

# 将两种动作的损失平均
actor_loss = (actor_loss_in + actor_loss_out) / 2
```

---

In [ ]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_in_dim, action_out_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3_in = torch.nn.Linear(hidden_dim, action_in_dim)
        self.fc3_out = torch.nn.Linear(hidden_dim, action_out_dim)

    def forward(self, x):
        x = F.relu(self.fc2(F.relu(self.fc1(x))))
        action_in_probs = F.softmax(self.fc3_in(x), dim=1)
        action_out_probs = F.softmax(self.fc3_out(x), dim=1)
        return action_in_probs, action_out_probs  # 返回两种动作的概率分布

#...[省略其他部分]

class PPO:
    #...[省略初始化部分]

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        action_in_probs, action_out_probs = self.actor(state)
        action_in_dist = torch.distributions.Categorical(action_in_probs)
        action_out_dist = torch.distributions.Categorical(action_out_probs)
        action_in = action_in_dist.sample()
        action_out = action_out_dist.sample()
        return action_in.item(), action_out.item()  # 返回两种动作

    def update(self, transition_dict):
        #...[省略前面部分]

        old_log_probs_in = torch.log(self.actor(states)[0].gather(1, actions[:, 0])).detach()
        old_log_probs_out = torch.log(self.actor(states)[1].gather(1, actions[:, 1])).detach()

        log_probs_in = torch.log(self.actor(states)[0].gather(1, actions[:, 0]))
        log_probs_out = torch.log(self.actor(states)[1].gather(1, actions[:, 1]))

        # Compute new r_t for both action types
        ratio_in = torch.exp(log_probs_in - old_log_probs_in)
        ratio_out = torch.exp(log_probs_out - old_log_probs_out)

        surr1_in = ratio_in * advantage
        surr2_in = torch.clamp(ratio_in, 1 - self.eps, 1 + self.eps) * advantage

        surr1_out = ratio_out * advantage
        surr2_out = torch.clamp(ratio_out, 1 - self.eps, 1 + self.eps) * advantage

        actor_loss_in = torch.mean(-torch.min(surr1_in, surr2_in))
        actor_loss_out = torch.mean(-torch.min(surr1_out, surr2_out))

        # 将两种动作的损失平均
        actor_loss = (actor_loss_in + actor_loss_out) / 2

#### ASN-QMIX
![ASN-QMIX](https://pic3.zhimg.com/80/v2-964f3eef79b37c99306f697ce3fe54b6_1440w.webp)

> 详情在QMIX后才会学到

#### 直观对比：
![](https://pic2.zhimg.com/80/v2-2d270c8402049556ba08466b1ff58189_1440w.webp)

### HPN超网络实现PI/PE

提出了一种新的网络结构——超策略网络（Hyper Policy Network，HPN），该网络结构能够满足 **置换不变性（permutation invariance，PI）** 和 **置换同变性（permutation equivariance，PE）** ，可与任意多智能体强化学习算法结合，最终性能超过了当前的许多算法

#### 背景
当前MARL面临严重的 **维度灾难（curse of dimensionality）**，因而中心思想就是利用 **置换不变性（permutation invariance，PI）** 和 **置换同变性（permutation equivariance，PE）** 的 _归纳偏置（inductive bias）_ 减小MARL算法中的状态-动作空间

**归纳偏置（inductive bias）**:
可以理解为，利用先验知识对模型做一定的约束，从而可以起到 “模型选择” 的作用

---

置换不变性（permutation invariance，PI）&置换同变性（permutation equivariance，PE）,不满足PI特性和PE特性统称为 **置换敏感（permutation-sensitive）** 函数

- PI置换不变性：f(x,y)=f(y,x)=(m,n) 即为置换输入元素排列不影响
- PE置换同变性: f(x,y)=(m,n) f(y,x)=(n,m)即为置换输入元素排列，会以同样方式置换输出排列

观测空间维度爆炸

每个智能体$i$的观测$o_i$包含了$m$个实体的特征，即有$[x_1, \cdots , x_m],x \in \mathcal{X} $，则考虑所有的组合，观测空间的维度有$\mathcal{X}^m$，然而很多时候 **不同排列对应向量所包含的信息都是相同的**，比如$[A,B,C]$和$[C,A,B]$实际上是相同的，意味着m个实体，可以把观测维度从排列的$m!\rightarrow \frac{1}{m!}$，因而需要构造PI神经网络

#### 如何利用PI/PE使网络高效
- 输入 $o_{i}$ 包含两类实体: 同盟 $o_{i}^{\text {ally }}$ 与敌人 $o_{i}^{\text {enemy }}$
- 输出包含两类动作的 $Q$ 值: 移动 $\mathcal{A}_{i}^{\text {move }}$ ，攻击 $\mathcal{A}_{i}^{\text {attack }}$

这时候应该满足:
- 变换 $o_{i}$ 中元素的排列， $\mathcal{A}_{i}^{\text {move }}$ 的 $Q$ 值不应该发生变化

    - 这意味着，不论你如何重排观察中的实体（即不论敌人和同伴的顺序如何），移动动作的预期回报都应该保持不变
    - 我们可以利用 \mathrm{PI} 模块使得 $Q_{i}\left(\mathcal{A}_{i}^{\text {move }} \mid o_{i}\right)$ 
- $o_{i}^{\text {enemy }}$ 的置换应该使得 $\mathcal{A}_{i}^{\text {attack }}$ 发生相同的置换

    - 而对于 $Q_{i}\left(\mathcal{A}_{i}^{\text {enemy }} \mid o_{i}\right)$ ，于 $o_{i}^{\text {enemy }}$ 中每个enemy的特征与 $\mathcal{A}_{i}^{\text {attack }}$ 中每个攻击的动作之间存在一一对应关系
    - 这表示攻击动作的 $Q$ 值与特定的敌人直接关联。例如，如果你的目标是攻击特定的敌人，那么你的动作选择将直接依赖于那个敌人的特征
    - 我们可以利用PE模块使得 $Q_{i}\left(\mathcal{A}_{i}^{\text {attack }} \mid o_{i}\right)$ 更加高效
    
---

#### 模型大致框架
![PI](https://pic3.zhimg.com/80/v2-c885363edd156ddc25d56db00bd053b2_1440w.webp)<br>
$A_i={A_{equiv},A_{inv}}$其中动作划分为：实体相关动作集合$A_{equiv}$，实体不相关动作集合$A_{inv}$，其中前者排序必须与$o_i$相同，而后者必须保持不变

该框架中，A-输入模块，B-主要模块(该模块可以是任意结构)，C-动作$a_{inv}$对应输出模块，D-动作$a_{equiv}$对应输出模块

输出关系为:$ z_{i}=A\left(o_{i}\right), h_{i}=B\left(z_{i}\right), \pi_{i}\left(a_{\text {inv }} \mid o_{i}\right)=C\left(h_{i}\right) ,\pi_{i}\left(a_{\text {equiv }} \mid o_{i}\right)=D\left(h_{i}\right) $

主要是对单个智能体的局部策略网络进行了设计，具体地，首先提出:
-  动态置换网络（Dynamic Permutation Network，DPN）
-  超策略网络（Hyper Policy Network，HPN）

---

#### DPN动态置换网络
![DPN](https://pic1.zhimg.com/80/v2-5ad94ea8b6195d0563d8d5755b39a2e0_1440w.webp)
大多数MARL中，ACD都是MLP全连接层，模块B则会引入RNN处理局部观测的输入

其中input为$m$个实体的观测$o_i$,而A被拆分为多个权重矩阵$\mathcal{W}_{\text {in }}=\left[\mathbf{W}_{1}^{\text {in }}, \ldots \mathbf{W}_{m}^{\text {in }}\right]$。

##### PI输入层
当$o_i$发生置换时，为了保持结果不变，也必须要对权重矩阵进行置换。这里是通过增加一个 **权重策略选择网络**，为每一个$o_i$输出一个权重矩阵的序号prob，进行softmax并取独热编码，就能实现PI：
$$
z_{i}=\sum_{j=1}^{m} o_{i}^{\prime}[j]\left(\hat{p}_{\text {in }}\left(o_{i}^{\prime}[j]\right) \mathcal{W}_{\text {in }}\right) \quad o_{i}^{\prime}=g o_{i}, \forall g \in G
$$
其中 $\hat{p}_{\text {in }}\left(o_{i}^{\prime}[j]\right)$是权重策略选择网络的独热编码，$W_{in}$为$\mathcal{W}_{\text {in }}=\left[\mathbf{W}_{1}^{\text {in }}, \ldots \mathbf{W}_{m}^{\text {in }}\right]$

展开写输出维度为m的权重选择网络:
$$
p_{\text {in }}\left(\left[\mathbf{W}_{1}^{\text {in }}, \ldots \mathbf{W}_{m}^{\text {in }}\right] \mid o_{i}[j]\right)=\operatorname{softmax}\left(\operatorname{MLP}\left(o_{i}[j]\right)\right)
$$
表明$o_i[j]$选择$W_{in}$中对应权重矩阵的概率，为了使得权重选择可导，采用Gumbel-Softmax操作得到argmax的索引one-hot编码

##### PE输出层
注意，在输出层仍然需要用到观测空间的信息，为了在输入排列的变化会造成输出排列发生相同变化，同样构造一个权重选择网络$\hat{p}_{\text {out }}\left(o_{i}[j]\right)$，其中D的j个输出可以计算为:
$$
\pi_{i}\left(a_{\text {equiv }} \mid o_{i}^{\prime}\right)[j]=h_{i}\left(\hat{p}_{\text {out }}\left(o_{i}^{\prime}[j]\right) \mathcal{W}_{\text {out }}\right) \quad o_{i}^{\prime}=g o_{i}, \forall g \in G, \forall j \in\{1, \ldots, m\}
$$
其中$h_i$为B的输出

> 通俗解释「从矩阵计算角度理解」请参看原文，这里不再赘述

---

### HPN超策略网络
![HPN](https://pic3.zhimg.com/80/v2-a25c9c3be37130ec8073e5bf945df0f6_1440w.webp)<br>
DPN的核心思想是总是为每个$o_i[j]$分配相同的权重矩阵，而权重矩阵只能从有限的参数集$W_{in}$和$W_{out}$中选择

那么可以使用 **超策略网络（hyper policy network，HPN）**，即直接利用一系列 **超网络（hypernetworks）**产生每个$o_i[j]$所对应的权重矩阵，因而表征能力更强

对于输入层，构造超网络$\operatorname{hp}_{\text {in }}\left(o_{i}[j]\right)=\operatorname{MLP}\left(o_{i}[j]\right)$，则公式为:
$$
z_{i}=\sum_{j=1}^{m} o_{i}^{\prime}[j] \mathrm{hp}_{\text {in }}\left(o_{i}^{\prime}[j]\right) \quad o_{i}^{\prime}=g o_{i}, \forall g \in G
$$

对于输出层，构造超网络$\operatorname{hp}_{\text {out }}\left(o_{i}[j]\right)=\operatorname{MLP}\left(o_{i}[j]\right)$，则公式为:
$$
\pi_{i}\left(a_{\text {equiv }} \mid o_{i}^{\prime}\right)[j]=h_{i} \mathrm{hp}_{\text {out }}\left(o_{i}^{\prime}[j]\right) \quad o_{i}^{\prime}=g o_{i}, \forall g \in G, \forall j \in\{1, \ldots, m\}
$$
由于每个$o_i[j]$都能产生独一无二的权重矩阵，参数空间大大增加，因而表征能力更强

---

#### 最终数据比较:
> 结论：HPN＞DPN≥UPDeT≥ASN>QMIX>GNN≈SET≈DA

- HPN的性能优于DPN，这是因为HPN通过超网络扩大了参数空间，因而表征能力更强；
- UPDeT、ASN能够达到与DPN相近的性能，说明PI特性与PE特性能够提升性能；
- GNN与SET的性能比原始QMIX更差（存疑），这俩结构实现了PI，但是没有实现PE，（我的理解是PI能够缩小状态空间，而PE则能够缩小动作空间），作者给出的解释是共享嵌入层的使用限制了模型表征能力；
- DA的性能不如原始QMIX，这说明用置换敏感的函数（MLP）来实现PI是十分困难的。